In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab

from cis import CIS

In [2]:
mlab.init_notebook()

Notebook initialized with ipy backend.


In [3]:
def evaluate_gaussian_basis(g, x, y, z):
    i, j, k = g.ijk
    x0, y0, z0 = g.A
    g_val = 0.0
    for n, d, alpha in zip(g.norm_const, g.coefs, g.exps):
        g_val += n * d * (x - x0)**i * (y - y0)**j * (z - z0)**k \
            * np.exp(-alpha * ((x - x0)**2 + (y - y0)**2 + (z - z0)**2))
    return g_val

In [4]:
mol = CIS()
mol.read_from_xyz('ethene.xyz')
mol.get_basis('sto-3g')

mol.initialize()
e_cis = mol.run_cis(nprint=20)

-110.65678762984008
CIS:
Excited State   1: E_exc =     3.7647 eV
7a   -> 8b      -0.683147 (46.7 %)
7b   -> 8a       0.724720 (52.5 %)

Excited State   2: E_exc =     3.7647 eV
7a   -> 8b       0.724728 (52.5 %)
7b   -> 8a       0.683155 (46.7 %)

Excited State   3: E_exc =     3.7647 eV
7a   -> 8a      -0.704240 (49.6 %)
7b   -> 8b       0.704240 (49.6 %)

Excited State   4: E_exc =    10.8304 eV
6a   -> 8b       0.479366 (23.0 %)
6b   -> 8a      -0.868098 (75.4 %)

Excited State   5: E_exc =    10.8304 eV
6a   -> 8b      -0.868098 (75.4 %)
6b   -> 8a      -0.479366 (23.0 %)

Excited State   6: E_exc =    10.8304 eV
6a   -> 8a      -0.701208 (49.2 %)
6b   -> 8b       0.701208 (49.2 %)

Excited State   7: E_exc =    10.9078 eV
5a   -> 8a       0.690687 (47.7 %)
5b   -> 8b      -0.690687 (47.7 %)

Excited State   8: E_exc =    10.9078 eV
5b   -> 8a      -0.974071 (94.9 %)

Excited State   9: E_exc =    10.9078 eV
5a   -> 8b      -0.974071 (94.9 %)

Excited State  10: E_exc =    11.4372

In [5]:
def build_grid(xlim, ylim, zlim, nx, ny, nz):
    x_ = np.linspace(*xlim, nx)
    y_ = np.linspace(*ylim, ny)
    z_ = np.linspace(*zlim, nz)
    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')
    return x, y, z

In [6]:
def evaluate_mo_grid(mol, grid):
    x, y, z = grid
    
    # Build MO coefficient matrix in spin-orbit basis
    n_spatial_orb = len(mol.mo_energy)
    c_spin = np.zeros((n_spatial_orb * 2, n_spatial_orb * 2))
    c_spin[:n_spatial_orb:, ::2] = mol.mo_coeff
    c_spin[n_spatial_orb:, 1::2] = mol.mo_coeff
    
    # Evaluate AOs on the grid
    ao_grid = np.zeros((n_spatial_orb * 2, NX, NY, NZ))
    
    for i, g in enumerate(mol.basisfunctions):
        g_grid = evaluate_gaussian_basis(g, x, y, z)
        ao_grid[i] = g_grid
        ao_grid[i + n_spatial_orb] = g_grid
    
    # Transform to MOs
    mo_grid = np.einsum('ij,ixyz->jxyz', c_spin, ao_grid)
    
    return mo_grid

In [7]:
R_VDW = {
    'H': 2.0787,
    'C': 3.2125,
    'N': 2.9291,
    'O': 2.8724,
}

R_COV = {
    'H': 0.6047,
    'C': 1.4173,
    'N': 1.3417,
    'O': 1.1905,
}

ATOM_COLOR = {
    'H': (0.8, 0.8, 0.8),
    'C': (0.0, 1.0, 0.0),
    'N': (0.0, 0.0, 1.0),
    'O': (1.0, 0.0, 0.0),
}

In [8]:
a = np.array([1,2,3])
b = np.array([4,5,6])

In [9]:
def visualize_cube(mol, grid, density, isovalues, colors, figure):
    # Draw atoms
    for a in mol.atomlist:
        p = mlab.points3d(
            *a.coord, R_VDW[a.symbol], color=ATOM_COLOR[a.symbol], 
            scale_factor=0.5, figure=figure,
        )
    
    # Draw bonds
    for i, a in enumerate(mol.atomlist):
        ca = a.coord
        sa = a.symbol
        ra = R_COV[sa]
        for b in mol.atomlist[:i]:
            cb = b.coord
            sb = b.symbol
            rb = R_COV[sb]
            
            vab = cb - ca
            rab = np.linalg.norm(vab)
            if rab < (ra + rb) * 1.2:
                mid = ca + vab * (ra / (ra + rb))
                p = mlab.plot3d(*np.vstack((ca, mid)).T, tube_radius=0.2, 
                                color=ATOM_COLOR[sa], figure=figure)
                p = mlab.plot3d(*np.vstack((cb, mid)).T, tube_radius=0.2, 
                                color=ATOM_COLOR[sb], figure=figure)

    # Draw isosurface
    for ival, c in zip(isovalues, colors):
        p = mlab.contour3d(*grid, density, color=c, contours=[ival], 
                           figure=figure)
    
    return p

In [10]:
XLIM, YLIM, ZLIM = (0.0, 20.0), (0.0, 20.0), (0.0, 20.0)
NX, NY, NZ = 80, 80, 80

grid = build_grid(XLIM, YLIM, ZLIM, NX, NY, NZ)
mo_grid = evaluate_mo_grid(mol, grid)

In [11]:
ISOSURFACE_COLORS = [(1, 0, 0), (0, 0, 1)]
ORBITAL_ISOVALUES = [0.05, -0.05]
DENSITY_ISOVALUES = [0.01, -0.01]

ihomo = (mol.nocc - 1) * 2
ilumo = mol.nocc * 2

In [ ]:
fig_homo = mlab.figure()
visualize_cube(mol, grid, mo_grid[ihomo], ORBITAL_ISOVALUES, ISOSURFACE_COLORS, fig_homo)
# mlab.show()

In [ ]:
fig_lumo = mlab.figure()
visualize_cube(mol, grid, mo_grid[ilumo], ORBITAL_ISOVALUES, ISOSURFACE_COLORS, fig_lumo)

In [13]:
ISTATE = 10

In [14]:
td_grid = np.zeros((NX, NY, NZ))
for (exc, c_ia) in zip(mol.excitations, mol.cis_states[:, ISTATE]):
    i, a = exc
    if (i % 2) == (a % 2):
        td_grid += c_ia * mo_grid[i] * mo_grid[a]

In [15]:
fig_td = mlab.figure()
visualize_cube(mol, grid, td_grid, DENSITY_ISOVALUES, ISOSURFACE_COLORS, fig_td)
# mlab.show()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…